In [102]:
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [56]:
code_files_all = ["relax-sequential.c", "relax-sequential-vec.c", "relax-omp.c", 
                    "relax-omp-vec.c", "relax-mpi.c", "relax-mpi-vec.c"]
code_files_seq = ["relax-omp.c", "relax-omp-vec.c", "relax-mpi.c", "relax-mpi-vec.c"]

def parse_matrix_sizes():
    with open('slurm-output.txt') as file:
        file_contents = file.read()
        lines = file_contents.split("\n");
        
        df_sizes = {}
        
        for file in code_files_all:
            df_sizes[file] = pd.DataFrame(columns=["Matrix size", "Number of cores", "Average runtime"])
            
        line_index = 0
        while line_index < len(lines):
            if "Science Cluster" in lines[line_index]:
                return df_sizes
            elif "Matrix size" in lines[line_index]:
                #Extract the matrix size
                size = lines[line_index].split(":")[1][:-1]
                #print(size)
                
                line_index += 1
                
                cores = 0
                
                while cores != 32:
                    #Extract number of cores
                    while not "Number of cores:" in lines[line_index]:
                        line_index += 1
                    cores = int(''.join(x for x in lines[line_index] if x.isdigit()))
                    line_index += 1

                    #Extract times from all 5 measurements
                    times = {}
                    measurement = 1
                    while measurement < 5:
                        while not "Measurement" in lines[line_index]:
                            line_index += 1
                        measurement = int(''.join(x for x in lines[line_index] if x.isdigit()))
                        line_index += 1
                        #Extract individual times:
                        if cores == 1:
                            for file in code_files_all:
                                while not file in lines[line_index]:
                                    line_index += 1
                                time = int(''.join(x for x in lines[line_index] if x.isdigit()))
                                if file in times.keys():
                                    times[file].append(time)
                                else:
                                    times[file] = [time]
                        else:
                            for file in code_files_seq:
                                while not file in lines[line_index]:
                                    line_index += 1
                                time = int(''.join(x for x in lines[line_index] if x.isdigit()))
                                if file in times.keys():
                                    times[file].append(time)
                                else:
                                    times[file] = [time]

                    #print(times)
                    if cores == 1:
                        for file in code_files_all:
                            df_sizes[file].loc[len(df_sizes[file].index)] = [size,cores,statistics.median(times[file])]
                    else:
                        for file in code_files_seq:
                            df_sizes[file].loc[len(df_sizes[file].index)] = [size,cores,statistics.median(times[file])]
            else:
                line_index += 1

def parse_iterations():
    with open('slurm-output.txt') as file:
        file_contents = file.read()
        lines = file_contents.split("\n");

        df_iterations = {}
        
        for file in code_files_all:
            df_iterations[file] = pd.DataFrame(columns=["Matrix size", "Number of cores", "Average runtime"])

        line_index = 0
        while line_index < len(lines):
            if "Science Cluster" in lines[line_index]:
                return df
            elif "Iterations" in lines[line_index]:
                #Extract the iterations
                iterations = int(''.join(x for x in lines[line_index] if x.isdigit()))
                
                if iterations == 900:
                    return df
                
                line_index += 1
                
                cores = 0
                
                while cores != 32:
                    #Extract number of cores
                    while not "Number of cores:" in lines[line_index]:
                        line_index += 1
                    cores = int(''.join(x for x in lines[line_index] if x.isdigit()))
                    line_index += 1

                    #Extract times from all 5 measurements
                    times = {}
                    measurement = 1
                    while measurement < 5:
                        while not "Measurement" in lines[line_index]:
                            line_index += 1
                        measurement = int(''.join(x for x in lines[line_index] if x.isdigit()))
                        line_index += 1
                        #Extract individual times:
                        if cores == 1:
                            for file in code_files_all:
                                while not file in lines[line_index]:
                                    line_index += 1
                                    if(line_index == len(lines)):
                                        print(iterations)
                                time = int(''.join(x for x in lines[line_index] if x.isdigit()))
                                if file in times.keys():
                                    times[file].append(time)
                                else:
                                    times[file] = [time]
                        else:
                            for file in code_files_seq:
                                while not file in lines[line_index]:
                                    line_index += 1
                                time = int(''.join(x for x in lines[line_index] if x.isdigit()))
                                if file in times.keys():
                                    times[file].append(time)
                                else:
                                    times[file] = [time]

                    #print(times)
                    if cores == 1:
                        for file in code_files_all:
                            df_iterations[file].loc[len(df_iterations[file].index)] = [size,cores,statistics.median(times[file])]
                    else:
                        for file in code_files_seq:
                            df_iterations[file].loc[len(df_iterations[file].index)] = [size,cores,statistics.median(times[file])]
            else:
                line_index += 1
                
df_sizes = parse_matrix_sizes()
#df_iterations = parse_iterations()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #print(df_sizes)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #print(df_iterations)
      

In [106]:
#for df in df_sizes:
    #plt.figure()
    #df_sizes[df].plot(kind='line', y="Average runtime", x="Matrix size", ylim=(0,15000))
    #df_sizes[df]["Average runtime"].plot(kind='line')
    #plt.show()
    #print(df)
    #print(df_sizes[df])
#print(df_sizes["relax-sequential.c"])
#print(df_sizes["relax-omp.c"])
    
print("Speedup OMP:")
speedup = pd.DataFrame(columns=df_sizes["relax-sequential.c"]["Matrix size"].values, index=[1,2,4,8,16,32])
df_seq = df_sizes["relax-sequential.c"]
df_par = df_sizes["relax-omp.c"]
for size in speedup.columns:
    time_seq = df_seq[(df_seq["Matrix size"] == size)]["Average runtime"].values[0]
    for cores in speedup[size].index:
        time_par = df_par[(df_par["Matrix size"] == size) & (df_par["Number of cores"] == cores)]["Average runtime"].values[0]
        speedup[size][cores] = "{:.2f}".format(time_seq / time_par)
display(HTML(speedup.to_html()))

print("Efficiency OMP:")
efficiency = pd.DataFrame(columns=df_sizes["relax-sequential.c"]["Matrix size"].values, index=[1,2,4,8,16,32])
for size in efficiency.columns:
    for cores in efficiency[size].index:
        efficiency[size][cores] = "{:.2f}".format((float(speedup[size][cores]) / cores) * 100)
        
display(HTML(efficiency.to_html()))

print("Speedup MPI:")
speedup = pd.DataFrame(columns=df_sizes["relax-sequential.c"]["Matrix size"].values, index=[1,2,4,8,16,32])
df_par = df_sizes["relax-mpi.c"]
for size in speedup.columns:
    time_seq = df_seq[(df_seq["Matrix size"] == size)]["Average runtime"].values[0]
    for cores in speedup[size].index:
        time_par = df_par[(df_par["Matrix size"] == size) & (df_par["Number of cores"] == cores)]["Average runtime"].values[0]
        if time_par == 0:
            time_par += 0.1
        speedup[size][cores] = "{:.2f}".format(time_seq / time_par)
display(HTML(speedup.to_html()))

print("Efficiency MPI:")
efficiency = pd.DataFrame(columns=df_sizes["relax-sequential.c"]["Matrix size"].values, index=[1,2,4,8,16,32])
for size in efficiency.columns:
    for cores in efficiency[size].index:
        efficiency[size][cores] = "{:.2f}".format((float(speedup[size][cores]) / cores) * 100)
        
display(HTML(efficiency.to_html()))

Speedup OMP:


,256 * 256,512 * 512,1024 * 1024,2048 * 2048,4096 * 4096,8192 * 8192,16384 * 16384,32768 * 32768
1,2.00,1.07,1.04,1.04,1.04,1.04,1.03,1.02
2,2.00,1.45,1.77,1.56,1.90,1.88,2.00,1.89
4,3.00,2.29,2.89,2.64,3.29,3.72,3.93,3.91
8,6.00,5.33,3.93,3.71,5.75,6.62,7.22,7.28
16,0.46,1.14,2.62,4.33,6.21,7.60,8.22,8.03
32,0.07,0.18,0.69,1.53,5.04,8.01,11.14,10.37


Efficiency OMP:


,256 * 256,512 * 512,1024 * 1024,2048 * 2048,4096 * 4096,8192 * 8192,16384 * 16384,32768 * 32768
1,200.00,107.00,104.00,104.00,104.00,104.00,103.00,102.00
2,100.00,72.50,88.50,78.00,95.00,94.00,100.00,94.50
4,75.00,57.25,72.25,66.00,82.25,93.00,98.25,97.75
8,75.00,66.62,49.12,46.38,71.88,82.75,90.25,91.00
16,2.88,7.12,16.38,27.06,38.81,47.50,51.38,50.19
32,0.22,0.56,2.16,4.78,15.75,25.03,34.81,32.41


Speedup MPI:


,256 * 256,512 * 512,1024 * 1024,2048 * 2048,4096 * 4096,8192 * 8192,16384 * 16384,32768 * 32768
1,1.50,1.14,0.98,0.77,0.86,0.80,0.80,0.84
2,3.00,2.00,1.90,1.43,1.72,1.49,1.44,1.49
4,6.00,2.67,3.44,2.40,2.96,2.71,2.76,2.53
8,6.00,5.33,4.23,2.84,4.42,3.34,4.51,4.61
16,60.00,8.00,6.88,5.20,6.91,7.77,5.15,6.91
32,0.40,0.89,6.11,5.78,7.15,7.96,6.43,6.73


Efficiency MPI:


,256 * 256,512 * 512,1024 * 1024,2048 * 2048,4096 * 4096,8192 * 8192,16384 * 16384,32768 * 32768
1,150.00,114.00,98.00,77.00,86.00,80.00,80.00,84.00
2,150.00,100.00,95.00,71.50,86.00,74.50,72.00,74.50
4,150.00,66.75,86.00,60.00,74.00,67.75,69.00,63.25
8,75.00,66.62,52.88,35.50,55.25,41.75,56.38,57.63
16,375.00,50.00,43.00,32.50,43.19,48.56,32.19,43.19
32,1.25,2.78,19.09,18.06,22.34,24.88,20.09,21.03
